In [ ]:
import cv2
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
from libs.common import get_images_in_dir
from libs.calibration import CameraCalibrator
# from libs.find_lanes import find_new_fit, get_both_fitx, get_road
from libs.polynom_finder import  plot_poly, PolynomFinder

### Calibrate

In [ ]:
calibration_data_dir = 'camera_cal/'

calibrator = CameraCalibrator(9, 6)

calibrate_images = get_images_in_dir(calibration_data_dir)
used_images = []

for image in calibrate_images:
    if calibrator.calibrate(image):
        used_images.append(image)


calibrator.finish_calibrate()

### Process video

In [ ]:
# test_images_dir = 'test_images/'
# def dump_image(image):
#     mpimg.imsave(test_images_dir + "failed3.jpg", image)
#     print("dumped")

In [ ]:
input_dir = "videos/"
output_dir = "output_videos/"

def process_file(video, finder=None):
    if finder is None:
        finder = PolynomFinder(calibrator)

    video_in = input_dir + video
    video_out = output_dir + video

    clip1 = VideoFileClip(video_in)
    white_clip = clip1.fl_image(finder.process_image)  # NOTE: this function expects color images!!
    %time
    white_clip.write_videofile(video_out, audio=False)

    return HTML("""
        <video width="960" height="540" controls>
          <source src="{0}">
        </video>
        """.format(video_out))

In [ ]:
left = [5.619491003016991e-06, -0.022389297191894888, 207.78001585426344] 
right = [-0.00017266842079864529, 0.13416494883492175, 1101.124026129575]
pf = PolynomFinder(calibrator, left, right)
process_file("project_video.mp4", pf)

In [ ]:
# Find new init poly as average from previous
def get_aver(vals):
    dist = vals.max() - vals.min()
    mean = vals.mean()
    return vals[np.abs((vals - mean)) < 0.25 * dist].mean()

def get_mean(polys):
    polys = np.array(polys)
    print()
    return [get_aver(polys[:, i]) for i in range(3)]

print(get_mean(pf.lefts), get_mean(pf.rights))

In [ ]:
# fig, axes = plt.subplots(len(pf.stat), 1, figsize=(5 * len(pf.stat), 15))
# for ax, (key, vals) in zip(axes, pf.stat.items()):
#     ax.plot(vals, label=key)
#     ax.legend()

In [ ]:
# left = [4.812023863075211e-05, -0.03276168724055028, 219.99970304556533]
# right = [-2.293796189315271e-05, 0.050311695862751006, 1071.6601760916587]
# pf = PolynomFinder(calibrator, left, right)
# process_file("challenge_video.mp4", pf)
# # print(get_mean(pf.lefts), get_mean(pf.rights))

In [ ]:
# left = [5.619491003016991e-06, -0.022389297191894888, 207.78001585426344] 
# right = [-0.00017266842079864529, 0.13416494883492175, 1101.124026129575]
# pf = PolynomFinder(calibrator, left, right)
# process_file("harder_challenge_video.mp4",pf)
# # print(get_mean(pf.lefts), get_mean(pf.rights))